Revisión de duplicados

In [ ]:
import os
import shutil
from PIL import Image
import imagehash
from collections import defaultdict
from tqdm import tqdm  # Barra de progreso

def format_path(path):
    return os.path.normpath(path)

# Rutas de los directorios principales (cada uno tiene 2 subdirectorios)
dir1 = format_path(r"C:\Users\Marc\Desktop\UNI\TFG\DATABASE_build\Sources\MAIN_DATASET\divided_dataset\BalancedDatabase\divided_dataset_original\test")
dir2 = format_path(r"C:\Users\Marc\Desktop\UNI\TFG\DATABASE_build\Sources\MAIN_DATASET\divided_dataset\BalancedDatabase\divided_dataset_original\train")
dir3 = format_path(r"C:\Users\Marc\Desktop\UNI\TFG\DATABASE_build\Sources\MAIN_DATASET\divided_dataset\BalancedDatabase\divided_dataset_original\val")
directorio_duplicados = format_path(r"C:\Users\Marc\Desktop\UNI\TFG\DATABASE_build\Sources\MAIN_DATASET\divided_dataset\BalancedDatabase\divided_dataset_original\duplicados")

# Crear carpeta de duplicados si no existe
os.makedirs(directorio_duplicados, exist_ok=True)

# Lista de directorios principales a analizar
directorios = [dir1, dir2, dir3]

# Diccionario para almacenar hashes y rutas de archivos
hash_dict = defaultdict(list)

# Función para calcular el hash perceptual de una imagen con tamaño 224x224
def calcular_hash(imagen_path):
    try:
        with Image.open(imagen_path) as img:
            img = img.convert("RGB")  # Convertir a RGB para evitar problemas de formato
            img = img.resize((224, 224))  # Redimensionar a 224x224
            return imagehash.phash(img)
    except Exception as e:
        print(f"⚠️ Error procesando {imagen_path}: {e}")
        return None

# Contar total de imágenes para la barra de progreso (incluyendo subdirectorios)
total_imagenes = sum(len(files) for d in directorios for _, _, files in os.walk(d))

# Escanear imágenes y almacenar hashes con tolerancia en la diferencia de hash
print("\n🔍 Escaneando imágenes...")
with tqdm(total=total_imagenes, desc="Procesando imágenes", unit="img") as pbar:
    for directorio in directorios:
        for subdir, _, archivos in os.walk(directorio):  # Busca en subdirectorios
            for archivo in archivos:
                ruta_completa = os.path.join(subdir, archivo)

                if os.path.isfile(ruta_completa) and archivo.lower().endswith(("jpg", "jpeg", "png")):
                    hash_imagen = calcular_hash(ruta_completa)
                    if hash_imagen is not None:
                        # Permitir una diferencia de hasta 5 en el hash para considerar imágenes similares
                        encontrado = False
                        for h in hash_dict.keys():
                            if hash_imagen - h <= 2:  # Tolerancia de 2
                                hash_dict[h].append((ruta_completa, subdir))
                                encontrado = True
                                break
                        if not encontrado:
                            hash_dict[hash_imagen].append((ruta_completa, subdir))

                    pbar.update(1)  # Actualizar barra de progreso

# Mover imágenes duplicadas y renombrarlas
contador_duplicados = 1  # Para identificar cada grupo de imágenes duplicadas
imagenes_movidas = 0

print("\n📂 Moviendo imágenes duplicadas...")
with tqdm(total=len(hash_dict), desc="Moviendo duplicados", unit="grupo") as pbar:
    for hash_valor, lista_imagenes in hash_dict.items():
        if len(lista_imagenes) > 1:  # Si hay más de una imagen con el mismo hash
            for idx, (ruta_original, subdirectorio_origen) in enumerate(lista_imagenes, start=1):
                nombre_original = os.path.basename(ruta_original)
                nombre_subdirectorio = os.path.basename(subdirectorio_origen)  # Obtener nombre del subdirectorio
                nuevo_nombre = f"dup{contador_duplicados}_{idx}_{nombre_subdirectorio}_{nombre_original}"
                nueva_ruta = os.path.join(directorio_duplicados, nuevo_nombre)

                # Mover la imagen renombrada a la carpeta duplicados
                shutil.move(ruta_original, nueva_ruta)
                imagenes_movidas += 1
                print(f"✔ Movida: {ruta_original} → {nueva_ruta}")

            contador_duplicados += 1  # Incrementar contador para el siguiente grupo de duplicados
        pbar.update(1)  # Actualizar barra de progreso

print(f"\n✅ Proceso completado. Se han movido {imagenes_movidas} imágenes duplicadas a '{directorio_duplicados}' 🚀")


Revisión de mismo color

In [ ]:
import os
import shutil
import cv2
import numpy as np
from tqdm import tqdm  # Importar tqdm para la barra de progreso

def is_single_color(image_path, tolerance):
    """Verifica si la imagen es aproximadamente de un solo color basándose en la desviación estándar."""
    # Leer la imagen en color
    image = cv2.imread(image_path)
    
    # Comprobar si la desviación estándar está por debajo del umbral de tolerancia
    return np.std(image) < tolerance

def move_single_color_images(directory, tolerance=15):
    """Identifica imágenes predominantemente de un solo color y las mueve a una carpeta 'color'."""
    if not os.path.exists(directory):
        print(f"El directorio {directory} no existe.")
        return

    # Directorio donde se almacenarán las imágenes de un solo color
    color_dir = os.path.join(directory, 'color')
    os.makedirs(color_dir, exist_ok=True)

    # Obtener la lista de archivos en el directorio
    file_list = [f for f in os.listdir(directory) if f.endswith('.jpg') or f.endswith('.png')]

    # Procesar imágenes con barra de progreso
    for filename in tqdm(file_list, desc="Procesando imágenes"):
        file_path = os.path.join(directory, filename)
        
        # Verificar si la imagen es predominantemente de un solo color
        if is_single_color(file_path, tolerance):
            shutil.move(file_path, os.path.join(color_dir, filename))
            print(f"Imagen de un solo color movida: {filename} a {color_dir}")

# Función para formatear rutas correctamente
def format_path(path):
    return os.path.normpath(path)

# Configuración de rutas
path_to_images = format_path(r"C:\Users\Marc\Desktop\UNI\TFG\DATABASE_build\Sources\MAIN_DATASET\All\malignant")

# Uso de la función
move_single_color_images(path_to_images, tolerance=15)


Detectar imagenes con pelo

In [ ]:
import os
import cv2
import numpy as np
import shutil

def detect_hair_in_image(image_path):
    # Leer la imagen
    img = cv2.imread(image_path)
    if img is None:
        return False

    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detectar bordes usando el algoritmo Canny
    edges = cv2.Canny(gray, threshold1=50, threshold2=150)

    # Aplicar un filtro para detectar áreas oscuras (cabello suele ser oscuro en muchas imágenes médicas)
    dark_mask = cv2.inRange(gray, 0, 70)

    # Combinamos los bordes y las áreas oscuras
    combined = cv2.bitwise_and(edges, dark_mask)

    # Contar los píxeles significativos en la imagen combinada
    count_non_zero = cv2.countNonZero(combined)

    # Si hay suficientes píxeles, asumimos que hay cabello en la imagen
    # Este umbral es ajustable según la sensibilidad deseada
    return count_non_zero > 3000

def move_images_with_hair(src_directory, dest_directory):
    if not os.path.exists(src_directory):
        print(f"The directory {src_directory} does not exist.")
        return

    os.makedirs(dest_directory, exist_ok=True)

    # Recorrer todas las imágenes en el directorio
    for filename in os.listdir(src_directory):
        file_path = os.path.join(src_directory, filename)

        # Saltar directorios u otros archivos que no sean imágenes (asumiendo .jpg y .png)
        if os.path.isdir(file_path) or not (filename.endswith('.jpg') or filename.endswith('.png')):
            continue

        # Detectar cabello en la imagen
        if detect_hair_in_image(file_path):
            # Mover la imagen a la carpeta de destino
            shutil.move(file_path, os.path.join(dest_directory, filename))
            print(f"Moved {filename} to {dest_directory}")

# Función para formatear rutas correctamente
def format_path(path):
    return os.path.normpath(path)

# Configuración de rutas
src_directory = format_path(r"C:\Users\Marc\Desktop\UNI\TFG\DATABASE_build\Sources\MAIN_DATASET\All\benign")
dest_directory = format_path(r"C:\Users\Marc\Desktop\UNI\TFG\DATABASE_build\Sources\MAIN_DATASET\All\benign\hair")

# Uso
move_images_with_hair(src_directory, dest_directory)


Division en conjuntos

In [ ]:
import os
import shutil
import random

def format_path(path):
    return os.path.normpath(path)

# Ruta de los directorios originales
original_dataset_dir = format_path(r"C:\Users\Marc\Desktop\UNI\TFG\DATABASE_build\Sources\MAIN_DATASET\All")
benign_dir = os.path.join(original_dataset_dir, 'benign')
malignant_dir = os.path.join(original_dataset_dir, 'malignant')

# Ruta de los directorios de destino
base_dir = format_path(r"C:\Users\Marc\Desktop\UNI\TFG\DATABASE_build\Sources\MAIN_DATASET\divided_dataset\BalancedDatabase\divided_dataset_original")
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Función para crear directorios si no existen
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Crear directorios principales
create_dir(train_dir)
create_dir(val_dir)
create_dir(test_dir)

# Crear subdirectorios para cada clase en cada conjunto
for subset in [train_dir, val_dir, test_dir]:
    create_dir(os.path.join(subset, 'benign'))
    create_dir(os.path.join(subset, 'malignant'))

# Función para dividir y copiar imágenes
def split_and_copy(class_name, src_dir, train_size, val_size, test_size):
    # Obtener lista de archivos
    filenames = os.listdir(src_dir)
    random.shuffle(filenames)
    
    total_images = len(filenames)
    
    # Calcular índices para división
    train_end = train_size
    val_end = train_size + val_size
    
    # Dividir archivos
    train_files = filenames[:train_end]
    val_files = filenames[train_end:val_end]
    test_files = filenames[val_end:train_size + val_size + test_size]
    
    # Copiar archivos
    for filename in train_files:
        src = os.path.join(src_dir, filename)
        dst = os.path.join(train_dir, class_name, filename)
        shutil.copyfile(src, dst)
        
    for filename in val_files:
        src = os.path.join(src_dir, filename)
        dst = os.path.join(val_dir, class_name, filename)
        shutil.copyfile(src, dst)
        
    for filename in test_files:
        src = os.path.join(src_dir, filename)
        dst = os.path.join(test_dir, class_name, filename)
        shutil.copyfile(src, dst)

# Dividir y copiar imágenes de la clase Benigna
split_and_copy(
    class_name='benign',
    src_dir=benign_dir,
    train_size=34623,
    val_size=7419,
    test_size=7420
)

# Dividir y copiar imágenes de la clase Maligna
split_and_copy(
    class_name='malignant',
    src_dir=malignant_dir,
    train_size=7028,
    val_size=1506,
    test_size=1507
)

print("División y copia completadas.")


Data Augmentation - Balanceada

In [ ]:
import os
import random
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

def format_path(path):
    return os.path.normpath(path)

# Ruta de los directorios originales
input_dir = format_path(r"C:\Users\Marc\Desktop\UNI\TFG\DATABASE_build\Sources\MAIN_DATASET\divided_dataset\BalancedDatabase\divided_dataset_original\train")
output_dir = format_path(r"C:\Users\Marc\Desktop\UNI\TFG\DATABASE_build\Sources\MAIN_DATASET\divided_dataset\BalancedDatabase\FinalDatabase_trainAugmented\train")

# Asegura de que el directorio de salida existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Clases
classes = ['benign', 'malignant']

# ------------------------- #
# FUNCIONES DE TRANSFORMACIÓN
# ------------------------- #
def rotate_with_zoom(img_cv, angle, zoom=1.0):
    height, width = img_cv.shape[:2]
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, zoom)
    rotated_img = cv2.warpAffine(
        img_cv, rotation_matrix, (width, height),
        flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE
    )
    return rotated_img

def adjust_brightness(img_cv, factor):
    hsv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2HSV).astype(np.float32)
    hsv[:, :, 2] *= factor
    hsv[:, :, 2] = np.clip(hsv[:, :, 2], 0, 255)
    hsv = hsv.astype(np.uint8)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

def adjust_contrast(img_cv, factor):
    lab = cv2.cvtColor(img_cv, cv2.COLOR_RGB2LAB).astype(np.float32)
    lab[:, :, 0] *= factor
    lab[:, :, 0] = np.clip(lab[:, :, 0], 0, 255)
    lab = lab.astype(np.uint8)
    return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

def horizontal_flip(img_cv):
    return cv2.flip(img_cv, 1)

def vertical_flip(img_cv):
    return cv2.flip(img_cv, 0)

def random_shift(img_cv, max_shift=0.2):
    h, w = img_cv.shape[:2]
    shift_x = random.uniform(-max_shift, max_shift) * w
    shift_y = random.uniform(-max_shift, max_shift) * h
    M = np.float32([[1, 0, shift_x],
                    [0, 1, shift_y]])
    shifted = cv2.warpAffine(
        img_cv, M, (w, h),
        flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE
    )
    return shifted

def random_blur(img_cv, max_ksize=3):
    ksize = random.choice([1, 3])
    if ksize == 1:
        return img_cv  # sin blur
    return cv2.GaussianBlur(img_cv, (ksize, ksize), 0)

# ------------------------- #
# LISTA / DICCIONARIO DE TRANSFORMACIONES POSIBLES
# ------------------------- #
def apply_random_transformation(img_cv):
    transformations = {
        'rotate_zoom': lambda img: rotate_with_zoom(
            img,
            angle=random.uniform(-45, 45),
            zoom=random.uniform(0.9, 1.1)
        ),
        'brightness': lambda img: adjust_brightness(
            img, factor=random.uniform(0.8, 1.2)
        ),
        'contrast': lambda img: adjust_contrast(
            img, factor=random.uniform(0.8, 1.2)
        ),
        'flipH': lambda img: horizontal_flip(img),
        'flipV': lambda img: vertical_flip(img),
        'shift': lambda img: random_shift(img, max_shift=0.2),
        'blur': lambda img: random_blur(img, max_ksize=3)
    }
    
    chosen_key = random.choice(list(transformations.keys()))
    transformed_img_cv = transformations[chosen_key](img_cv)
    return transformed_img_cv, chosen_key

# ------------------------- #
# PROCESAMIENTO DE IMÁGENES
# ------------------------- #
for cls in classes:
    class_input_dir = os.path.join(input_dir, cls)
    class_output_dir = os.path.join(output_dir, cls)
    if not os.path.exists(class_output_dir):
        os.makedirs(class_output_dir)

    print(f"Procesando clase: {cls}")

    image_files = [
        f for f in os.listdir(class_input_dir)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]

    for filename in tqdm(image_files, desc=f"Procesando {cls}", unit="imagen"):
        img_path = os.path.join(class_input_dir, filename)
        base_filename = os.path.splitext(filename)[0]
        
        # Leer imagen en RGB
        img_cv = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

        if cls == 'benign':
            # 1) Imagen original
            # 2) UNA transformación aleatoria
            transformations = [(img_cv, 'orig')]

            transformed_img, trans_name = apply_random_transformation(img_cv)
            transformations.append((transformed_img, trans_name))

        elif cls == 'malignant':
            # 1 imagen original + 9 transformaciones
            transformations = [(img_cv, 'orig')]
            for _ in range(9):
                transformed_img, trans_name = apply_random_transformation(img_cv)
                transformations.append((transformed_img, trans_name))

        # Guardar las imágenes transformadas
        for idx, (transformed_img_cv, trans_name) in enumerate(transformations):
            output_filename = f"{base_filename}_{idx+1}_{trans_name}.jpg"
            output_path = os.path.join(class_output_dir, output_filename)
            transformed_img_bgr = cv2.cvtColor(transformed_img_cv, cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_path, transformed_img_bgr)

# ------------------------- #
# BALANCEAR LAS CLASES
# ------------------------- #
print("Balanceando las clases...")
benign_dir = os.path.join(output_dir, 'benign')
malignant_dir = os.path.join(output_dir, 'malignant')

benign_images = [
    f for f in os.listdir(benign_dir)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
]
malignant_images = [
    f for f in os.listdir(malignant_dir)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
]

# Función que retorna sólo las imágenes 'augmentadas' (sin '_orig')
def get_augmented_images(file_list):
    return [f for f in file_list if "_orig" not in f]

# Al eliminar, nos aseguramos de borrar sólo las imágenes aumentadas
if len(benign_images) > len(malignant_images):
    # Exceso en benign
    excess_count = len(benign_images) - len(malignant_images)
    benign_aug = get_augmented_images(benign_images)  # sólo augmentations
    if len(benign_aug) >= excess_count:
        to_remove = random.sample(benign_aug, excess_count)
        for filename in to_remove:
            os.remove(os.path.join(benign_dir, filename))
            print(f"Eliminada imagen aumentada: {filename} de benign")
    else:
        # Si no hay suficientes augmentations para borrar, se borran todas las disponibles
        for filename in benign_aug:
            os.remove(os.path.join(benign_dir, filename))
            print(f"Eliminada imagen aumentada: {filename} de benign")
        print("No había suficientes augmentations para equilibrar totalmente, pero se han eliminado todas las disponibles.")

elif len(malignant_images) > len(benign_images):
    # Exceso en malignant
    excess_count = len(malignant_images) - len(benign_images)
    malignant_aug = get_augmented_images(malignant_images)  # sólo augmentations
    if len(malignant_aug) >= excess_count:
        to_remove = random.sample(malignant_aug, excess_count)
        for filename in to_remove:
            os.remove(os.path.join(malignant_dir, filename))
            print(f"Eliminada imagen aumentada: {filename} de malignant")
    else:
        # Si no hay suficientes augmentations para borrar, se borran todas las disponibles
        for filename in malignant_aug:
            os.remove(os.path.join(malignant_dir, filename))
            print(f"Eliminada imagen aumentada: {filename} de malignant")
        print("No había suficientes augmentations para equilibrar totalmente, pero se han eliminado todas las disponibles.")

print("Clases balanceadas.")

